In [28]:
import os
import pandas
import copy
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import xml.etree.cElementTree as et 



In [17]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 220;

<IPython.core.display.Javascript object>

In [18]:
ERROR_TYPES = {
    "1100": "correct", "1101": "correct",
    "1000": "incorrect", 
    "1001": "misapplied",
    "0000": "out of graph", "0001": "out of graph", "0100": "out of graph", "0101": "out of graph", 
    "1110": "correct repeat", 
    "1010": "repeat", "1011": "repeat", "1111": "repeat", 
    "0110": "where error", "0111": "where error",
    "0011": "when error", 
    "0010": "wild error"
}


In [19]:
def transaction_file_to_df(path):
    df = pandas.read_csv(path, sep='\t', lineterminator='\n', skip_blank_lines=True).replace({r'\r': ''}, regex=True)
    df = df.rename(index=int, columns={a:a.rstrip() for a in df.keys()})
    return df

In [20]:
def extract_from_brd(path):

    parsedXML = et.parse(path)
    dfcols = ["match_s","match_a","match_i","match_actor","match_ID","match_source","match_dest"]
    df_xml = pandas.DataFrame(columns= dfcols)

    for e in parsedXML.findall(".//edge"):    
        ID = e.find('./actionLabel/uniqueID').text
        source = e.find('./sourceID').text
        dest = e.find('./destID').text

        s = e.find('.actionLabel/matchers/Selection/matcher/matcherParameter').text
        a = e.find('.actionLabel/matchers/Action/matcher/matcherParameter').text
        i =  e.find('.actionLabel/matchers/Input/matcher/matcherParameter').text  
        actor = e.find('.actionLabel/matchers/Actor').text 
 
        df_xml = df_xml.append(
                pandas.Series([s, a, i, actor, ID, source, dest], index= dfcols),
                ignore_index=True)
        
        df_xml[['match_ID', 'match_source', 'match_dest']] = df_xml[['match_ID', 'match_source', 'match_dest']].apply(pandas.to_numeric)
    
    return(df_xml)
    




In [21]:
def update_input(row): 
    if(row['match_i'] == 'x'):
        return("I need to convert these fractions before solving.: true")
    
    if(row['match_i'] == 'v'):
        return("I need to convert these fractions before solving.: false")
    
    else:
        return(str(row['match_i']))
    
    
def update_action(row): 
    if(row['match_a'] == 'UpdateTextArea'):
        if(row['match_i'] == 'x' or row['match_i'] == 'v'): 
            return('UpdateCheckBox') 
          
        else: return('UpdateTextField') 
    if(row['match_a'] == 'ButtonPressed'): 
           return('ButtonPressed')
        
        
def update_selection(row): 
    if(row['match_s'] == 'done'): 
        return('done') 
    elif(row['match_s'][10] == '8'): 
        return('check_convert') 
    else: 
        if(row['match_s'][10] == '4'): 
            box = '3' 
        if(row['match_s'][10] == '5'): 
            box = '4' 
        if(row['match_s'][10] == '6'): 
            box = '5' 
        if(row['match_s'][13] == '0'): 
            pos = 'num' 
        if(row['match_s'][13] == '1'): 
            pos = 'den' 
        return(pos+box)
           

# Change variable names in CTAT interface to ones used by students
def clean_extract(extract):
    new = extract.copy()
    new.insert(0, 'match_i_new', np.nan)
    new.insert(0, 'match_a_new', np.nan)
    new.insert(0, 'match_s_new', np.nan)
    new['match_s_new'] = new.apply(update_selection, axis=1)
    new['match_a_new'] = new.apply(update_action, axis=1)
    new['match_i_new'] = new.apply(update_input, axis=1)
    return(new)


# No Tutor Performed SAIs
def get_correct_transaction(transaction):
    return (transaction[(transaction.Outcome == 'CORRECT') 
                & (transaction.Action != 'setVisible')
                & (transaction.Input != '+')])




#def graph_len(g):
#    begin = find_beginnings(g)
#    count = 1
#    i = (list(begin))[0]
#    while len(g[i]) !=0:
#        i = g[i][0]
#        count = count + 1
#    return(count)

    

# If transactions end incorrectly, return False 
def check_transactions(df, graph):
    if( (df['Outcome'].iloc[-1] == 'INCORRECT') | (df['Outcome'].iloc[-1] == 'HINT')): return(False)
    return(True)
  


In [22]:
# Remove correct steps in transactions that are not in the brds
def clean_transaction(df):
    r = -1
    for _, table_row in df.iterrows():
        
        if(table_row['Input'] == 'I need to convert these fractions before solving.: false' and table_row['Outcome'] == 'CORRECT'):
            r = _
    if r!= -1:
        df = df.drop([r])          
    return(df)




#Return a graph created from SAI_extract
def get_graph_from_extract(extract):
    graph = {}
    tl = set()
    for _,table_row in extract.iterrows():
        ID = table_row['match_ID']
        source = table_row['match_source']
        dest = table_row['match_dest']
        t = (ID, source, dest)
        tl.add(t)
        graph[t] = []
    for i in graph:
        d = i[2]
        for j in tl:
            if (j[1] == d):
                graph[i].append(j)
    return(graph)





In [23]:

#For human student's correct SAI, find possible ID, source and dest of all edges with same SAI (for the first node)
def get_first_correct_IDs(row, extract):
    lst = [] 
    S, A, I = row['Selection'], row['Action'], row['Input']
    for _,table_row in extract.iterrows():

        if(S == table_row['match_s_new'] and A == table_row['match_a_new'] and I == table_row['match_i_new']):
            ID, source, dest = table_row['match_ID'], table_row['match_source'], table_row['match_dest']
            lst.append((ID, source, dest))
    return(lst)



#def get_first_node(row, extract, graph):
#    lst = get_first_correct_IDs(row, extract)
#    for (ID, s, d) in lst:
#        first = True
#        for node in graph:
#            if (ID, s, d) in graph[node]:
#                first = False
#        if(first):
#            return((ID, s, d))
#    return(None)


# Dictionary of SAI for each unique node 
def get_node_SAIs(graph, cleaned_extract):
    correct_sai = {}
    for (ID, s, d) in graph:
        for _,table_row in cleaned_extract.iterrows():
            if(ID == table_row['match_ID'] and s == table_row['match_source'] and d == table_row['match_dest']):
                correct_sai[(ID, s, d)] = ((table_row['match_s_new'], table_row['match_a_new'], table_row['match_i_new']))
    return(correct_sai)


#Find all beginning nodes 
def find_beginnings(graph):
    s = set()
    for i in graph:
        start = True
        for j in graph:
            if i in graph[j]:
                start = False
        if start:
            s.add(i)           
    return(s)
   
    
    
def find_current_index(correct, incorrect):
    res = {}
    correct_copy = copy.deepcopy(correct)
    incorrect_copy = copy.deepcopy(incorrect)
    while (incorrect_copy and (incorrect_copy[-1] > correct_copy[-1])):
        res[incorrect_copy[-1]] = correct_copy[-1]
        incorrect_copy.pop()
    
    for i in range(len(incorrect_copy)):
        if (incorrect_copy[i] < correct_copy[0]):
            res[incorrect_copy[i]] = correct_copy[0]
        else:
            while(correct_copy and (incorrect_copy[i] > correct_copy[0])):
                correct_copy.pop(0)
            res[incorrect_copy[i]] = correct_copy[0]
    return(res)



def find_last_index(correct, incorrect):
    incorrect_copy = copy.deepcopy(incorrect)
    while (incorrect_copy and (incorrect_copy[0] < correct[0])):
        incorrect_copy.pop(0)
    res = {}
    inc, cor = incorrect_copy[::-1], correct[::-1]
    for i in inc:
        if(i < cor[0]):
            while(i < cor[0]):
                cor.pop(0)
            res[i] = cor[0]
        else:
            res[i] = cor[0]
    return(res)


# bfs
def search(graph, start):
    if start not in graph: return None
    visited, queue = [], [start]
    while queue:
        node = queue.pop(0)
        if node not in visited:
            visited.append(node)
            queue.extend(set(graph[node]) - set(visited))           
    return visited




def check_downstream(graph, up, down):
    if down in search(graph, up): return True
    return False 



#return first node that matches
def first_match(df, lst, d, last_node, graph):
    s_c, a_c, i_c = df['Selection'], df['Action'], df['Input']
    
    for node in lst:
        if check_downstream(graph, last_node, node):
            (s_l, a_l, i_l) = d[node] 
            if (s_c, a_c, i_c) ==  (s_l, a_l, i_l):
                return node          
    return None       




        
def match_steps(one_student, cleaned_extract, graph):
    match = one_student.copy()
    match['SAI'], match['node'], match['downstream'], match['d_nodes'] = None, None, None, None
    match['S_current'], match['I_current'], match['S_downstream'], match['I_downstream'] = None, None, None, None
    
    
    #Add new columns error types and KC to df
    match['KC_toward'], match['error_type'] = None, None
    
    
    correct = get_correct_transaction(match) 
    correct_queue = []
    all_first_nodes = find_beginnings(graph)
    sai_dict = get_node_SAIs(graph, cleaned_extract)
    
    for _,table_row in correct.iterrows():
        correct_queue.append([_, table_row])

        
    # Match correct steps:    
    # for loop because might start with multiple nodes in brd
    for begin in all_first_nodes:
        
        SAI_d = {}
        done = set()
        
        last_node = begin
        correct_order = search(graph, begin)
        correct_transaction = {}
        correct_queue_copy = copy.deepcopy(correct_queue)
        can_match = True
        
        
        while correct_queue_copy and correct_order:

            n = correct_queue_copy.pop(0)
            next_index, next_correct = n[0], n[1]
            
            s, a, i = next_correct['Selection'], next_correct['Action'], next_correct['Input']
            
            
            if (s,a,i) in done:
            #deal with duplications
            
                first_index = SAI_d[(s,a,i)]
                correct_transaction[next_index] = correct_transaction[first_index]
                
                
            
            else:
                done.add((s,a,i))
                SAI_d[(s,a,i)]= next_index
    
                f_match = first_match(next_correct, correct_order, sai_dict, last_node, graph)

            
                if f_match == None:
                    can_match = False
                    break
            
                
                while correct_order[0] != f_match:
                    correct_order.pop(0)
                correct_order.pop(0)
                    
                correct_transaction[next_index] = f_match
                last_node = f_match
        

        if can_match == True: break
            
    
    if can_match == False:  
        print("Can not match for this step_slice", match)
        return match
    
    
    else:
        
        index = sorted(list(correct_transaction.keys()))
       
        for i in index:
            match.at[i,'SAI'] = 'correct'
            match.at[i, 'node'] = correct_transaction[i]        
            match.at[i, 'S_current'], match.at[i, 'I_current'] = 1,1
       
        incorrect = list((match[match['Outcome'] == 'INCORRECT']).index)

        # match incorrect steps 
        if incorrect: 
    
            cur_index = find_current_index(index, incorrect)
            last_index = find_last_index(index, incorrect)


            for i in incorrect:
            
                match.at[i ,'SAI'] = 'incorrect'
                
                KC = match.at[cur_index[i], 'KC (Field)'] #KC of the node working towards 
                match.at[i, 'KC_toward'] = KC


                # Find current steps for incorrect transactions 
                if(i not in last_index.keys()):
                    match.at[i, 'node'] = list(find_beginnings(graph))

                if(i in last_index.keys()):

                    n_l = match.at[last_index[i], 'node'] 
                    match.at[i, 'node'] = graph[n_l]

                
                
                sel, inp = match.at[i, 'Selection'], match.at[i, 'Input']
                s_found_c, i_found_c = 0, 0
                
                match_s_c = None 
                for c in match.at[i, 'node']:
                    (s_c, a_c, i_c) = sai_dict[c]
                    if s_c == sel: 
                        s_found_c = 1
                        match_s_c = c
                        
                    if i_c == inp: i_found_c = 1
                match.at[i, 'S_current'], match.at[i, 'I_current'] = s_found_c, i_found_c
                        
                        
                downstream = set()
                # downstream for a particular branch
                if match_s_c != None:
                    down = graph[match_s_c]
                    for d in down: 
                        downstream.add(d)
                        
                else:
                
                    for node in match.at[i, 'node']:
                        down = graph[node]
                        for d in down: 
                            downstream.add(d)

                match.at[i, 'downstream'] = list(downstream)
                
                
                


                d_nodes = set()
                for n in match.at[i, 'downstream']:
                    d_nodes = d_nodes.union(search(graph, n))
     
                match.at[i, 'd_nodes'] = d_nodes
                s_found_d, i_found_d = 0, 0
                for dn in d_nodes:
                    (s_d, a_d, i_d) = sai_dict[dn]
                    if(sel == s_d):
                        s_found_d = 1    
                    if(inp == i_d):
                        i_found_d = 1

   
                match.at[i, 'S_downstream'], match.at[i, 'I_downstream']  = s_found_d, i_found_d
            

                e_type = str(s_found_c) + str(i_found_c) + str(s_found_d) + str(i_found_d) 
                match.at[i, 'error_type'] = ERROR_TYPES[e_type]
                     
            
        return match


 

In [24]:
# For cases that end in incorrect or hint, return the original df with new blank columns 
def original_df(one_student):

    match = one_student.copy()
    match['SAI'], match['node'], match['downstream'], match['d_nodes'] = None, None, None, None
    match['S_current'], match['I_current'], match['S_downstream'], match['I_downstream'] = None, None, None, None
    
     #Add error types and KC
    match['KC_toward'], match['error_type'] = None, None
    return match
    
    







In [25]:
def one_student_all_problems(df, directory, stu):
    
    new = pandas.DataFrame(columns = list(df.columns))
    new['SAI'], new['node'], new['downstream'], new['d_nodes'] = None, None, None, None
    new['S_current'], new['I_current'], new['S_downstream'], new['I_downstream'] = None, None, None, None
    
    new['KC_toward'], new['error_type'] = None, None
    
    
    for problem in df['Problem Name'].unique(): # preserve order
        stu_slice = ( df[df['Problem Name'] == problem] ).copy()
        
        stu_slice = clean_transaction(stu_slice)
        
     #   print("problem", problem)
        
        if (problem != "InstructionSlide"):
            brd = extract_from_brd(directory + "/" + problem + ".xml")
            graph = get_graph_from_extract(brd)

            if check_transactions(stu_slice, graph):
                tutor_SAI = clean_extract(brd)
                stu_match = match_steps(stu_slice, tutor_SAI, graph)
                new = new.append(stu_match)
                
            else:
                orig_match = original_df(stu_slice)
                new = new.append(orig_match)

    return(new)
    
    
    
    
    
    
    

In [26]:
def generate_truth_table_revised(transactions, brd_directory):
    t = transaction_file_to_df(transactions)

    df = pandas.DataFrame(columns = list(t.columns))
    df['SAI'], df['node'], df['downstream'], df['d_nodes'] = None, None, None, None
    df['S_current'], df['I_current'], df['S_downstream'], df['I_downstream'] = None, None, None, None
    
    df['KC_toward'], df['error_type'] = None, None
    
    
    for stu in t['Anon Student Id'].unique():
        p = t[t['Anon Student Id'] == stu]
        new = one_student_all_problems(p, brd_directory, stu)
        df = df.append(new)
            
    df.to_csv('/Users/daniellaye/Desktop/human_error_types.csv')       

    return df

In [27]:
error = generate_truth_table_revised("/Users/daniellaye/Desktop/data/human_converted_v2.txt", "/Users/daniellaye/Desktop/brds")

error







problem is InstructionSlide
problem is MD 1_2_times_2_3
problem is MD 1_3_times_1_6
problem is MD 1_3_times_3_2
problem is MS 7_9_times_4_9
problem is MD 1_3_times_4_9
problem is MD 3_2_times_1_4
problem is MD 3_4_times_1_8
problem is MS 1_7_times_3_7
problem is MD 3_7_times_2_8
problem is MD 4_5_times_2_7
problem is MD 4_5_times_3_10
problem is MS 2_9_times_3_9
problem is MD 5_4_times_1_2
problem is MD 5_6_times_1_7
problem is MD 5_6_times_2_3
problem is MS 3_8_times_7_8
problem is MD 5_9_times_3_7
problem is MD 6_7_times_1_6
problem is MD 7_8_times_1_4
problem is MS 2_6_times_3_6
problem is MD 7_10_times_2_5
problem is MD 11_9_times_2_3
problem is MD 2_3_times_1_2
problem is MS 11_2_times_3_2
problem is AS 1_7_plus_3_7
problem is AS 2_3_plus_4_3
problem is AS 2_5_plus_4_5
problem is AS 2_6_plus_3_6
problem is AS 2_9_plus_3_9
problem is AS 3_8_plus_7_8
problem is AS 5_4_plus_3_4
problem is AS 7_9_plus_4_9
problem is AS 11_2_plus_3_2
problem is AS 3_7_plus_4_7
problem is AD 1_2_plus_2_

problem is AS 11_2_plus_3_2
problem is AS 3_7_plus_4_7
problem is AD 1_2_plus_2_3
problem is AD 1_3_plus_3_2
problem is InstructionSlide
problem is AS 1_7_plus_3_7
problem is AD 5_3_plus_9_8
problem is AD 5_3_plus_2_5
problem is MD 4_5_times_3_10
problem is MD 4_5_times_2_7
problem is AS 2_3_plus_4_3
problem is AD 3_7_plus_2_8
problem is MD 1_3_times_1_6
problem is AS 2_6_plus_3_6
problem is MD 1_3_times_3_2
problem is AS 2_9_plus_3_9
problem is AD 4_9_plus_3_4
problem is AD 1_2_plus_2_3
problem is MS 1_7_times_3_7
problem is MD 3_7_times_2_8
problem is MD 1_3_times_4_9
problem is AD 4_5_plus_2_7
problem is AD 1_3_plus_3_2
problem is MD 1_2_times_2_3
problem is AS 2_5_plus_4_5
problem is MD 3_4_times_1_8
problem is MD 3_2_times_1_4
problem is MS 2_9_times_3_9
problem is MS 7_9_times_4_9
problem is MD 5_6_times_2_3
problem is MD 7_8_times_1_4
problem is MS 2_6_times_3_6
problem is AD 1_4_plus_4_5
problem is MS 3_8_times_7_8
problem is MS 11_2_times_3_2
problem is AS 11_2_plus_3_2
proble

problem is MD 3_2_times_1_4
problem is AD 5_3_plus_2_5
problem is MD 1_3_times_1_6
problem is MD 1_3_times_4_9
problem is AS 1_7_plus_3_7
problem is MD 1_3_times_3_2
problem is MS 1_7_times_3_7
problem is AD 1_3_plus_3_2
problem is AD 4_9_plus_3_4
problem is MD 3_7_times_2_8
problem is MS 2_9_times_3_9
problem is MD 1_2_times_2_3
problem is AS 2_5_plus_4_5
problem is AD 3_7_plus_2_8
problem is AD 4_5_plus_2_7
problem is AS 2_6_plus_3_6
problem is AS 2_9_plus_3_9
problem is MD 4_5_times_2_7
problem is MS 7_9_times_4_9
problem is AD 1_2_plus_2_3
problem is AS 2_3_plus_4_3
problem is MD 4_5_times_3_10
problem is AD 5_9_plus_3_7
problem is AS 11_2_plus_3_2
problem is MD 7_8_times_1_4
problem is MD 2_3_times_1_2
problem is AS 7_9_plus_4_9
problem is MD 7_10_times_2_5
problem is MD 11_9_times_2_3
problem is AS 3_7_plus_4_7
problem is MD 5_6_times_2_3
problem is MS 2_6_times_3_6
problem is AD 5_6_plus_5_7
problem is MD 6_7_times_1_6
problem is AS 3_8_plus_7_8
problem is MS 3_8_times_7_8
probl

problem is MD 5_6_times_2_3
problem is MS 3_8_times_7_8
problem is MD 5_9_times_3_7
problem is MD 6_7_times_1_6
problem is MD 7_8_times_1_4
problem is MS 2_6_times_3_6
problem is MD 7_10_times_2_5
problem is MD 11_9_times_2_3
problem is MD 2_3_times_1_2
problem is MS 11_2_times_3_2
problem is InstructionSlide
problem is MD 3_4_times_1_8
problem is MD 1_2_times_2_3
problem is AD 4_5_plus_2_7
problem is AS 2_5_plus_4_5
problem is MD 1_3_times_4_9
problem is MS 2_9_times_3_9
problem is MD 4_5_times_2_7
problem is AS 1_7_plus_3_7
problem is AS 2_9_plus_3_9
problem is MD 3_2_times_1_4
problem is MS 1_7_times_3_7
problem is MD 4_5_times_3_10
problem is AD 5_3_plus_2_5
problem is AD 3_7_plus_2_8
problem is MD 1_3_times_1_6
problem is AD 1_2_plus_2_3
problem is MS 7_9_times_4_9
problem is MD 3_7_times_2_8
problem is AD 5_3_plus_9_8
problem is MD 1_3_times_3_2
problem is AD 4_9_plus_3_4
problem is AD 7_8_plus_1_3
problem is AS 7_9_plus_4_9
problem is MS 3_8_times_7_8
problem is AS 5_4_plus_3_4


problem is MD 1_2_times_2_3
problem is AS 2_5_plus_4_5
problem is MD 3_2_times_1_4
problem is MD 4_5_times_3_10
problem is MS 7_9_times_4_9
problem is AD 4_9_plus_3_4
problem is InstructionSlide
problem is MD 1_3_times_3_2
problem is MD 1_3_times_4_9
problem is MS 2_9_times_3_9
problem is AD 4_9_plus_3_4
problem is MS 7_9_times_4_9
problem is AS 2_9_plus_3_9
problem is MD 4_5_times_2_7
problem is MD 3_7_times_2_8
problem is AD 5_3_plus_9_8
problem is MD 1_2_times_2_3
problem is AD 3_7_plus_2_8
problem is AS 2_3_plus_4_3
problem is MD 1_3_times_1_6
problem is AD 5_3_plus_2_5
problem is MD 4_5_times_3_10
problem is AS 1_7_plus_3_7
problem is MD 3_2_times_1_4
problem is AD 1_3_plus_3_2
problem is MD 3_4_times_1_8
problem is AD 4_5_plus_2_7
problem is MS 1_7_times_3_7
problem is AD 1_2_plus_2_3
problem is AS 2_5_plus_4_5
problem is AS 2_6_plus_3_6
problem is MD 5_6_times_2_3
problem is AD 1_4_plus_4_5
problem is AD 5_9_plus_3_7
problem is MD 7_8_times_1_4
problem is AS 7_9_plus_4_9
problem

...

problem is AS 3_8_plus_7_8
problem is AS 5_4_plus_3_4
problem is AS 7_9_plus_4_9
problem is AS 11_2_plus_3_2
problem is AS 3_7_plus_4_7
problem is AD 1_2_plus_2_3
problem is AD 1_3_plus_3_2
problem is AD 3_7_plus_2_8
problem is AD 4_5_plus_2_7
problem is AD 4_9_plus_3_4
problem is AD 5_3_plus_2_5
problem is AD 5_3_plus_9_8
problem is AD 5_6_plus_1_7
problem is AD 5_9_plus_3_7
problem is AD 6_7_plus_1_6
problem is AD 7_8_plus_1_3
problem is AD 5_6_plus_5_7
problem is AD 2_3_plus_3_5
problem is AD 1_4_plus_4_5
problem is MD 1_2_times_2_3
problem is MD 1_3_times_1_6
problem is MD 1_3_times_3_2
problem is MS 7_9_times_4_9
problem is MD 1_3_times_4_9
problem is MD 3_2_times_1_4
problem is MD 3_4_times_1_8
problem is MS 1_7_times_3_7
problem is MD 3_7_times_2_8
problem is MD 4_5_times_2_7
problem is MD 4_5_times_3_10
problem is MS 2_9_times_3_9
problem is MD 5_4_times_1_2
problem is MD 5_6_times_1_7
problem is MD 5_6_times_2_3
problem is MS 3_8_times_7_8
problem is MD 5_9_times_3_7
problem i

problem is AD 5_3_plus_9_8
problem is AD 5_6_plus_1_7
problem is AD 5_9_plus_3_7
problem is AD 6_7_plus_1_6
problem is AD 7_8_plus_1_3
problem is AD 5_6_plus_5_7
problem is AD 2_3_plus_3_5
problem is AD 1_4_plus_4_5
problem is InstructionSlide
problem is MD 1_2_times_2_3
problem is MD 1_3_times_1_6
problem is MD 1_3_times_3_2
problem is MS 7_9_times_4_9
problem is MD 1_3_times_4_9
problem is MD 3_2_times_1_4
problem is MD 3_4_times_1_8
problem is MS 1_7_times_3_7
problem is MD 3_7_times_2_8
problem is MD 4_5_times_2_7
problem is MD 4_5_times_3_10
problem is MS 2_9_times_3_9
problem is MD 5_4_times_1_2
problem is MD 5_6_times_1_7
problem is MD 5_6_times_2_3
problem is MS 3_8_times_7_8
problem is MD 5_9_times_3_7
problem is MD 6_7_times_1_6
problem is MD 7_8_times_1_4
problem is MS 2_6_times_3_6
problem is MD 7_10_times_2_5
problem is MD 11_9_times_2_3
problem is MD 2_3_times_1_2
problem is MS 11_2_times_3_2
problem is InstructionSlide
problem is MD 1_3_times_1_6
problem is MD 4_5_times_

problem is MD 1_3_times_4_9
problem is AD 5_3_plus_9_8
problem is MD 1_3_times_3_2
problem is AS 1_7_plus_3_7
problem is MS 1_7_times_3_7
problem is AD 3_7_plus_2_8
problem is AD 1_2_plus_2_3
problem is AD 5_3_plus_2_5
problem is MD 3_4_times_1_8
problem is MD 4_5_times_3_10
problem is MD 3_7_times_2_8
problem is MD 1_2_times_2_3
problem is MD 1_3_times_1_6
problem is MD 2_3_times_1_2
problem is AD 5_9_plus_3_7
problem is MS 2_6_times_3_6
problem is AD 5_6_plus_1_7
problem is MD 7_8_times_1_4
problem is MD 11_9_times_2_3
problem is AS 3_8_plus_7_8
problem is AS 3_7_plus_4_7
problem is MS 11_2_times_3_2
problem is AS 5_4_plus_3_4
problem is MS 3_8_times_7_8
problem is AS 7_9_plus_4_9
problem is AD 7_8_plus_1_3
problem is MD 5_4_times_1_2
problem is MD 5_6_times_1_7
problem is MD 5_6_times_2_3
problem is AD 6_7_plus_1_6
problem is MD 5_9_times_3_7
problem is AS 11_2_plus_3_2
problem is MD 7_10_times_2_5
problem is MD 6_7_times_1_6
problem is AD 5_6_plus_5_7
problem is AD 1_4_plus_4_5
pro

problem is MD 7_8_times_1_4
problem is MS 2_6_times_3_6
problem is MD 7_10_times_2_5
problem is MD 11_9_times_2_3
problem is MD 2_3_times_1_2
problem is MS 11_2_times_3_2
problem is AS 1_7_plus_3_7
problem is AS 2_3_plus_4_3
problem is AS 2_5_plus_4_5
problem is AS 2_6_plus_3_6
problem is AS 2_9_plus_3_9
problem is AS 3_8_plus_7_8
problem is AS 5_4_plus_3_4
problem is AS 7_9_plus_4_9
problem is AS 11_2_plus_3_2
problem is AS 3_7_plus_4_7
problem is AD 1_2_plus_2_3
problem is AD 1_3_plus_3_2
problem is AD 3_7_plus_2_8
problem is AD 4_5_plus_2_7
problem is AD 4_9_plus_3_4
problem is AD 5_3_plus_2_5
problem is AD 5_3_plus_9_8
problem is AD 5_6_plus_1_7
problem is AD 5_9_plus_3_7
problem is AD 6_7_plus_1_6
problem is AD 7_8_plus_1_3
problem is AD 5_6_plus_5_7
problem is AD 2_3_plus_3_5
problem is AD 1_4_plus_4_5


,Row,Sample Name,Transaction Id,Anon Student Id,Session Id,Time,Time Zone,Duration (sec),Student Response Type,Student Response Subtype,...,SAI,node,downstream,d_nodes,S_current,I_current,S_downstream,I_downstream,KC_toward,error_type
1,1262,All Data,e4b2d8fbbc7d4965285f8c5fea4ac764,Stu_01266dfb27cc2e1a087884753dbe4f67,S2d09ff826c4a57c89367e5261ac59ff4,2015-11-04 14:21:17,America/New_York,18,ATTEMPT,NaN,...,correct,"(7, 1, 5)",None,None,1,1,None,None,None,None
2,1263,All Data,7f67589b71f0c22267c5c47c0a4d81e2,Stu_01266dfb27cc2e1a087884753dbe4f67,S2d09ff826c4a57c89367e5261ac59ff4,2015-11-04 14:21:32,America/New_York,8,ATTEMPT,NaN,...,correct,"(9, 5, 3)",None,None,1,1,None,None,None,None
3,1264,All Data,6e9e408dd8a2320dd4469c14d27a989e,Stu_01266dfb27cc2e1a087884753dbe4f67,S2d09ff826c4a57c89367e5261ac59ff4,2015-11-04 14:21:32,America/New_York,8,ATTEMPT,NaN,...,correct,"(5, 3, 4)",None,None,1,1,None,None,None,None
4,1265,All Data,66a8b2dfac4f949097adb6b520207a72,Stu_01266dfb27cc2e1a087884753dbe4f67,S2d09ff826c4a57c89367e5261ac59ff4,2015-11-04 14:21:43,America/New_York,7,ATTEMPT,NaN,...,correct,"(7, 1, 5)",None,None,1,1,None,None,None,None
5,1266,All Data,b67dd082a6d43685d346e87c329e371d,Stu_01266dfb27cc2e1a087884753dbe4f67,S2d09ff826c4a57c89367e5261ac59ff4,2015-11-04 14:21:46,America/New_York,2,ATTEMPT,NaN,...,correct,"(9, 5, 3)",None,None,1,1,None,None,None,None
6,1267,All Data,093cbe779c72f9bafddf1905d99068f3,Stu_01266dfb27cc2e1a087884753dbe4f67,S2d09ff826c4a57c89367e5261ac59ff4,2015-11-04 14:21:46,America/New_York,2,ATTEMPT,NaN,...,correct,"(5, 3, 4)",None,None,1,1,None,None,None,None
7,1268,All Data,81735dd7538a9872acfdc0962a34803a,Stu_01266dfb27cc2e1a087884753dbe4f67,S2d09ff826c4a57c89367e5261ac59ff4,2015-11-04 14:21:56,America/New_York,7,ATTEMPT,NaN,...,correct,"(7, 1, 5)",None,None,1,1,None,None,None,None
8,1269,All Data,fff6750339c1e210f6bb52963bfaf4c6,Stu_01266dfb27cc2e1a087884753dbe4f67,S2d09ff826c4a57c89367e5261ac59ff4,2015-11-04 14:21:58,America/New_York,1,ATTEMPT,NaN,...,correct,"(9, 5, 3)",None,None,1,1,None,None,None,None
9,1270,All Data,1d350ff32fd43d1344f99f074dff497b,Stu_01266dfb27cc2e1a087884753dbe4f67,S2d09ff826c4a57c89367e5261ac59ff4,2015-11-04 14:21:58,America/New_York,1,ATTEMPT,NaN,...,correct,"(5, 3, 4)",None,None,1,1,None,None,None,None
10,1271,All Data,211055aac76af110bbd5f1f90b646e0c,Stu_01266dfb27cc2e1a087884753dbe4f67,S2d09ff826c4a57c89367e5261ac59ff4,2015-11-04 14:22:07,America/New_York,5,ATTEMPT,NaN,...,correct,"(7, 1, 5)",None,None,1,1,None,None,None,None
